In [18]:
from typing import Dict, Any
from collections import defaultdict

def convert_to_cytoscape_format(linking_output: Dict[str, Any]) -> Dict[str, Any]:
    nodes_seen = set()
    nodes = []
    edges = []

    for item in linking_output["linked_premises"]:
        premise = item["premise_text"]
        claim = item["linked_claim_text"]
        stance = item["stance"]

        if claim is None:
            continue  # Skip unlinked premises

        # Add premise node (if not already added)
        if premise not in nodes_seen:
            nodes.append({
                "data": {
                    "id": premise,
                    "type": "premise",
                    "stance": stance
                }
            })
            nodes_seen.add(premise)

        # Add claim node (if not already added)
        if claim not in nodes_seen:
            nodes.append({
                "data": {
                    "id": claim,
                    "type": "claim",
                    "stance": "neutral"
                }
            })
            nodes_seen.add(claim)

        # Add edge from premise to claim
        edges.append({
            "data": {
                "source": premise,
                "target": claim,
                "label": stance
            }
        })

    return {"nodes": nodes, "edges": edges}


In [26]:
!wget https://raw.githubusercontent.com/Horizontal-Labs/synapse-sparks/refs/heads/main/output.json -O output.json

--2025-07-04 22:47:41--  https://raw.githubusercontent.com/Horizontal-Labs/synapse-sparks/refs/heads/main/output.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1133 (1.1K) [text/plain]
Saving to: ‘output.json’

output.json         100%[===================>]   1.11K  --.-KB/s    in 0s      

2025-07-04 22:47:41 (36.5 MB/s) - ‘output.json’ saved [1133/1133]



In [28]:
import json

with open("output.json") as f:
    linked_dict = json.load(f)

# The same converter you've used before
def convert_to_cytoscape_format(linking_output):
    nodes = []
    edges = []
    nodes_seen = set()

    for item in linking_output["linked_premises"]:
        premise = item["premise_text"]
        claim = item["linked_claim_text"]
        stance = item["stance"]

        if claim is None:
            continue  # Skip unlinked premises

        if premise not in nodes_seen:
            nodes.append({
                "data": {
                    "id": premise,
                    "type": "premise",
                    "stance": stance
                }
            })
            nodes_seen.add(premise)

        if claim not in nodes_seen:
            nodes.append({
                "data": {
                    "id": claim,
                    "type": "claim",
                    "stance": "neutral"
                }
            })
            nodes_seen.add(claim)

        edges.append({
            "data": {
                "source": premise,
                "target": claim,
                "label": stance
            }
        })

    return {"nodes": nodes, "edges": edges}

cytoscape_data = convert_to_cytoscape_format(linked_dict)


In [29]:
html_template = f"""
<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8">
  <title>Argument Graph</title>
  <script src="https://unpkg.com/cytoscape@3.21.1/dist/cytoscape.min.js"></script>
  <style>
    #cy {{
      width: 100%;
      height: 95vh;
      display: block;
      border: 1px solid #ccc;
    }}
  </style>
</head>
<body>
  <h2 style="text-align:center;">Argument Graph (Claims and Premises)</h2>
  <div id="cy"></div>

  <script>
    const data = {json.dumps(cytoscape_data, indent=2)};

    const stanceColor = {{
      pro: '#4CAF50',
      con: '#F44336',
      neutral: '#2196F3'
    }};

    const cy = cytoscape({{
      container: document.getElementById('cy'),
      elements: data,
      style: [
        {{
          selector: 'node',
          style: {{
            'background-color': function(ele) {{ return stanceColor[ele.data('stance')]; }},
            'label': 'data(id)',
            'shape': function(ele) {{ return ele.data('type') === 'claim' ? 'ellipse' : 'roundrectangle'; }},
            'text-wrap': 'wrap',
            'text-max-width': 200,
            'text-valign': 'center',
            'text-halign': 'center',
            'font-size': 14,
            'color': '#fff',
            'text-outline-width': 2,
            'text-outline-color': '#333',
            'padding': '10px',
            'width': function(ele) {{ return Math.max(100, ele.data('id').length * 6); }},
            'height': function(ele) {{
              const lines = Math.ceil(ele.data('id').length / 25);
              return lines * 25;
            }}
          }}
        }},
        {{
          selector: 'edge',
          style: {{
            'curve-style': 'bezier',
            'target-arrow-shape': 'triangle',
            'line-color': function(ele) {{ return stanceColor[ele.data('label')]; }},
            'target-arrow-color': function(ele) {{ return stanceColor[ele.data('label')]; }},
            'label': 'data(label)',
            'font-size': 11,
            'color': '#000',
            'text-wrap': 'wrap',
            'text-background-opacity': 1,
            'text-background-color': '#fff',
            'text-background-padding': '4px',
            'text-margin-y': -5
          }}
        }}
      ],
      layout: {{
        name: 'cose',
        padding: 30
      }}
    }});
  </script>
</body>
</html>
"""

# Save the HTML to file
with open("graph.html", "w") as f:
    f.write(html_template)


In [30]:
from google.colab import files
files.download("graph.html")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>